In [72]:
import pandas as pd
import numpy as np 
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img,img_to_array
print(tf.__version__)

2.13.0


In [73]:
train_dir = Path('train')
train_filepaths = list(train_dir.glob(r'**/*.jpg'))

In [74]:
print(train_filepaths)

[WindowsPath('train/apple/Image_1.jpg'), WindowsPath('train/apple/Image_10.jpg'), WindowsPath('train/apple/Image_16.jpg'), WindowsPath('train/apple/Image_17.jpg'), WindowsPath('train/apple/Image_18.jpg'), WindowsPath('train/apple/Image_19.jpg'), WindowsPath('train/apple/Image_2.jpg'), WindowsPath('train/apple/Image_20.jpg'), WindowsPath('train/apple/Image_21.jpg'), WindowsPath('train/apple/Image_23.jpg'), WindowsPath('train/apple/Image_24.jpg'), WindowsPath('train/apple/Image_25.jpg'), WindowsPath('train/apple/Image_26.jpg'), WindowsPath('train/apple/Image_27.jpg'), WindowsPath('train/apple/Image_28.jpg'), WindowsPath('train/apple/Image_3.jpg'), WindowsPath('train/apple/Image_31.jpg'), WindowsPath('train/apple/Image_32.jpg'), WindowsPath('train/apple/Image_33.jpg'), WindowsPath('train/apple/Image_34.jpg'), WindowsPath('train/apple/Image_36.jpg'), WindowsPath('train/apple/Image_37.jpg'), WindowsPath('train/apple/Image_38.jpg'), WindowsPath('train/apple/Image_39.jpg'), WindowsPath('train

In [ ]:
test_dir = Path('test')
test_filepaths = list(test_dir.glob(r'**/*.jpg'))

In [ ]:
val_dir = Path('validation')
val_filepaths = list(val_dir.glob(r'**/*.jpg'))

In [ ]:
 str(train_filepaths[1]).split("\\")

In [ ]:
def image_processing(filepath):

    
    labels = [str(filepath[i]).split("\\")[-2] \
              for i in range(len(filepath))]
    
    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')
    
    df = pd.concat([filepath, labels], axis=1)
    
    df = df.sample(frac=1).reset_index(drop=True)
    
    return df

In [ ]:
train_df = image_processing(train_filepaths)
test_df = image_processing(test_filepaths)
val_df = image_processing(val_filepaths)

In [ ]:
print(f'Number of pictures: {train_df.shape[0]}\n')
print(f'Number of different labels: {len(train_df.Label.unique())}\n')
print(f'Labels {train_df.Label.unique()}')

In [ ]:
train_df.head(5)

In [ ]:
df_unique = train_df.copy().drop_duplicates(subset=["Label"]).reset_index()

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(7, 6), 
                         subplot_kw={'xticks': [],'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(df_unique.Filepath[i]))
    ax.set_title(df_unique.Label[i], fontsize = 12)
plt.tight_layout(pad=0.5)
plt.show()

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
val_images = train_generator.flow_from_dataframe(
    dataframe=val_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
pretrained_model.trainable = False

In [ ]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(36, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    batch_size = 32,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=2,
            restore_best_weights=True
        )
    ]
)

In [ ]:
# Predict the label of the test_images
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)
# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred1 = [labels[k] for k in pred]
pred1

In [ ]:
def output(location):
    img=load_img(location,target_size=(224,224,3))
    img=img_to_array(img)
    img=img/255
    img=np.expand_dims(img,[0])
    answer=model.predict(img)
    print(answer)
    print(answer.max())
    print("bre")
    y_class = answer.argmax(axis=-1)
    print(y_class)
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = labels[y]
    return res

In [ ]:
img = output('img23.jpg')
import matplotlib.image as mpimg
img2 = mpimg.imread('img23.jpg')
plt.imshow(img2)
print(img)

In [ ]:
tf.keras.models.save_model(model, 'model.pbtxt')

converter = tf.lite.TFLiteConverter.from_keras_model(model=model)

model_tflite = converter.convert()

open("GrocRecogModel.tflite","wb").write(model_tflite)

In [ ]:
import tensorflow as tf
test_loss, test_accuracy = model.evaluate(test_images, verbose=1)

print(f'Test accuracy: {test_accuracy}')